In [ ]:
import okex.Account_api as Account
import okex.Funding_api as Funding
import okex.Market_api as Market
import okex.Public_api as Public
import okex.Trade_api as Trade
import okex.subAccount_api as SubAccount
import okex.status_api as Status
import json
import numpy as np
import asyncio 
import websockets
import requests
import dateutil.parser as dp
from dateutil import tz
import hmac
import base64
import zlib
import datetime
import pandas as pd
import time
import nest_asyncio
import math
from decimal import Decimal, getcontext, setcontext,ROUND_DOWN, ROUND_UP,ROUND_CEILING
nest_asyncio.apply()

def get_timestamp():
    now = datetime.datetime.now()
    t = now.isoformat("T", "milliseconds")
    return t + "Z"

time_1 = get_timestamp()

def precfloat(num,digi):
    return int(round(float(num)*math.pow(10,digi)))/math.pow(10,digi)

In [ ]:
def xiadan_list(mode,Best_Number,*args,**kwargs):    
    P_rate=mode['P_rate']
    MarketOrder_arrive_average_volume=mode['MarketOrder_arrive_average_volume']
    limit_volume=mode['limit_volume']
    

    Tick_size=mode['Tick_size']
    new_volume5_arr=mode['new_volume5_arr']
    new_price5=mode['new_price5']
    opp_new_price5=mode['opp_new_price5']
    bidask_index=mode['bidask_index']    #会等于1或-1,影响Price_0的价格
    Tick_digit=mode['Tick_digit']     #交易价格到的最小位数
    ask_bid_pricegap=mode['ask_bid_pricegap']
    niddle_priceorder_times=mode['niddle_priceorder_times']
    TargetOrder_Deliver0=mode['TargetOrder_Deliver0']
    TargetOrder_Deliver1=mode['TargetOrder_Deliver1']
    #TargetOrder_Deliver2=mode['TargetOrder_Deliver2']
    #TargetOrder_Deliver3=mode['TargetOrder_Deliver3']
    
    for i in range(len(new_price5)):
        if new_volume5_arr[:i+1].sum()>max(TargetOrder_Deliver1/P_rate,2*MarketOrder_arrive_average_volume):  
            Price_1=precfloat(new_price5[i]-bidask_index*niddle_priceorder_times*Tick_size,Tick_digit)    #ab-->ask/bid
           #Price_1=precfloat(new_price5[i]-bidask_index*niddle_priceorder_times*Tick_size,Tick_digit)    #ab-->ask/bid
            if i==0:#主力挂单在1档
                #print("i==0")
                if abs(opp_new_price5[i]-new_price5[i]) > ask_bid_pricegap*Tick_size:       #2可以是一个变数<<<<<<       
                    Price_0=precfloat(new_price5[i]-bidask_index*niddle_priceorder_times_Price_0*Tick_size,Tick_digit)                         
                    
      #              TargetOrder_Deliver0=limit_volume
                else:
                    Price_0=precfloat(new_price5[i]-bidask_index*niddle_priceorder_times_Price_0*Tick_size,Tick_digit)  
                    #print('bid_Price_0_真零',bid_Price_0)
                    #print('TargetOrder_Deliver0',TargetOrder_Deliver0)
                #Price_2=precfloat(new_price5[i+1]-bidask_index*niddle_priceorder_times*Tick_size,Tick_digit)    #1可以是一个变数
                #print('bid_Price_2',bid_Price_2)
                #print('TargetOrder_Deliver2',TargetOrder_Deliver2)
                #Price_3=Price_2-bidask_index*niddle_priceorder_times*Tick_size   #5可以是一个变数
                #print('bid_Price_3',bid_Price_3)
                #print('TargetOrder_Deliver3',TargetOrder_Deliver3)


            elif i==Best_Number-1:#主力在最后一档
                #print('i=4')
                Price_0=precfloat(new_price5[i]-bidask_index*niddle_priceorder_times_Price_0*Tick_size,Tick_digit)  
                #print('bid_Price_0_i4',bid_Price_0)
    #            TargetOrder_Deliver0=limit_volume
                #print('TargetOrder_Deliver0',TargetOrder_Deliver0)
                #Price_2=Price_1-bidask_index*niddle_priceorder_times*Tick_size                   #5可以是一个变数
                #print('bid_Price_2_',bid_Price_2)
                #print('TargetOrder_Deliver2',TargetOrder_Deliver2)
                #Price_3=''
                #TargetOrder_Deliver3=''
            else:
                #print('i=1~3')
                Price_0=precfloat(new_price5[i]-bidask_index*niddle_priceorder_times_Price_0*Tick_size,Tick_digit)         
                #print('bid_Price_0_i1到3',bid_Price_0)
      #          TargetOrder_Deliver0=limit_volume
                #print('TargetOrder_Deliver0',TargetOrder_Deliver0)
                #Price_2=precfloat(new_price5[i+1]-bidask_index*niddle_priceorder_times*Tick_size,Tick_digit)         
                #print('bid_Price_2',bid_Price_2)
                #print('TargetOrder_Deliver2',TargetOrder_Deliver2)
                #Price_3=precfloat(Price_2-bidask_index*niddle_priceorder_times*Tick_size,Tick_digit)       #5可以是一个变数
                #print('bid_Price_3',bid_Price_3)
                #print('TargetOrder_Deliver3',TargetOrder_Deliver3)
            break

        else:
            if i==Best_Number-1:
                #print("前五档量都不够")
                Price_1=precfloat(new_price5[i]-bidask_index*niddle_priceorder_times*Tick_size,Tick_digit)    #ab-->ask/bid
               # Price_1=precfloat(new_price5[i]-bidask_index*niddle_priceorder_times*Tick_size,Tick_digit)    #此时这里是float
                Price_0=precfloat(new_price5[i]-bidask_index*niddle_priceorder_times_Price_0*Tick_size,Tick_digit)                    
     #           TargetOrder_Deliver0=limit_volume             
                #Price_2=Price_1-bidask_index*niddle_priceorder_times*Tick_size                      #5可以是一个变数
                #Price_3=''
                #TargetOrder_Deliver3=0                
    return [[Price_0,TargetOrder_Deliver0],[Price_1,TargetOrder_Deliver1]] 

def Uncomplete_order_update(mode):
    Trading_swap_list=mode['Trading_swap_list']   
    res=mode['trading_order_update_info']
    uncomplete=mode["result_uncomplete_order"]
    for i in Trading_swap_list:
    #用来承接某instrument_id的list
        for j in res['data']:
            if j['instrument_id']== i:     #整理出特定的合约种类  j['instrument_id']== i
                for y in range(len(uncomplete[i])): 
                    
                    if j['order_id']==uncomplete[i][y]['order_id']:
                        #print(uncomplete[i][y]['order_id'])#uncomplete['DOGE-USDT-SWAP']
                        if j['state']=='2' or j['state']=='4' or j['state']=='-1':
                            del uncomplete[i][y]
                        elif j['state']=='1'or j['state']=='0':
                            uncomplete[i][y]=j
                if j['state']=='3' or j['state']=='0':
                    uncomplete[i].append(j)
    return uncomplete

def bid_ask_mode_switch(BAI,mode):
    newmode={}
    for i in mode:        
        if len(mode[i])==2:
            if BAI=="bid":
                newmode[i]=mode[i][0]
            elif BAI=='ask':
                newmode[i]=mode[i][1]
        else :
            newmode[i]=mode[i]
    return newmode

def close_position_judge(mode,result_uncomplete_order): 
    if len(result_uncomplete_order['data'])!=0:
        for i in result_uncomplete_order['data'][0]:  #表示有遗留单
            if i['label']==mode['label']:   #找label
                pass
            else:
                break
            if i['price']==mode['opp_bid_ask'][0][0]:    #是否是best单 
                pass
            else:
                break                                   #无法判断是否是唯一的单,bit.com没有提供订单个数的查询
            mode['AGG_position_close_mode']='on'    #3个条件都满足,新五档是1～5
            if mode['bid_ask'][0][0]==str(mode['begin_price5'][0]):   #best bid没有改变
                mode['cancel_mode_type']='off'
    return mode['AGG_position_close_mode'],mode['cancel_mode_type']


def Indentify_Position_Premiunm(Result_Position,instrument_id,quantity_limit,BAGMODE,AAGMODE,bid_mode,ask_mode,base):  
    
    if Result_Position[0]['details']!=[] :   #2是因为永续合约有多空两个方向，交割合约可能有很多个
       # holding_position = float(Result_Position[0]['pos'])        
        #持仓差
       # holding_position= abs(float(Result_Position[0]['details'][0]['eq'])-base)
        
        if float(Result_Position[0]['details'][0]['cashBal'])>=base:
            #现货高于base,需要平多
            long_holding_position=float(Result_Position[0]['details'][0]['cashBal'])-base
            LSdiff=abs(long_holding_position)            
            
            if quantity_limit> long_holding_position >quantity_limit*0.5:
                ask_mode='on'         #AAG是平多               
            
            elif long_holding_position>=quantity_limit:
                AAGMODE='on'
                bid_mode='off'                
                
            else:  #long_holding_position <= quantity_limit*0.4                
                AAGMODE='off'
                LSdiff=0
                
        elif float(Result_Position[0]['details'][0]['cashBal'])<base:
            #现货低于base,买多
            short_holding_position=base-float(Result_Position[0]['details'][0]['cashBal']) 
            LSdiff=abs(short_holding_position)
            if quantity_limit>abs(short_holding_position)>quantity_limit*0.5:
                      
                bid_mode='on'
            elif abs(short_holding_position)>=quantity_limit:
                BAGMODE='on'             #平空   
                ask_mode='off'
                #print("AAGMODE=on")
            else:
                BAGMODE='off'  
                LSdiff=0
                
    elif float(Result_Position[0]['details']['cashBal'])==0:
        LSdiff=0
        bid_mode='on'
        ask_mode='on'
      
        
        
    return BAGMODE,AAGMODE,bid_mode,ask_mode,abs(LSdiff)

def precfloat(num,digi):
    return int(round(float(num)*math.pow(10,digi)))/math.pow(10,digi)

In [ ]:
if __name__ == '__main__':
    api_key = ""
    secret_key = ""
    passphrase = ""
    # flag是实盘与模拟盘的切换参数
    # flag = '1'  # 模拟盘
    flag = '0'  # 实盘
    
    publicAPI = Public.PublicAPI(api_key, secret_key, passphrase, False, flag)
    accountAPI = Account.AccountAPI(api_key, secret_key, passphrase, False, flag)
    tradeAPI = Trade.TradeAPI(api_key, secret_key, passphrase, False, flag)
    marketAPI = Market.MarketAPI(api_key, secret_key, passphrase, False, flag)

    spot_instrument_id='DOGE-USDT'
    use_currency='DOGE'
    

In [ ]:
P_rate_cap=0.1                     #预期参与率最大比例
Target_amount=100              #想建的最大仓位
Time_cycle=10                      #时间的检查周期
TolerancePosition=5
Time_full_round=480#预计花多少时间建仓，预设8小时
T_left=5#剩多少时间，用本地的先
       
        
spot_info = publicAPI.get_instruments('SPOT')['data']
for i in spot_info:
    if i['instId'] == spot_instrument_id:
        Tick_size = float(i["tickSz"])
        Tick_digit = np.log10(1 / float(i["tickSz"]))
        min_size = float(i['minSz'])


Best_Number=5  #取几档

niddle_priceorder_times_Price_0=110  #影响price_0的价格,距离best价有几个tick_size
niddle_priceorder_times= 140
limit_volume=800  #最低下多少
base=0

'''需要统计的参数'''
MarketBuyOrder_arrive_average_volume=10#每一单的平均量
MarketBuyOrder_arrive_rate_1min=30#此数值要用统计取得,单位是单数

market_trade_volume=1
#max(limit_volume,P_rate*MarketOrder_arrive_average_volume)
TargetOrder_Deliver0=int(limit_volume)
TargetOrder_Deliver1=int(limit_volume*2)
#TargetOrder_Deliver2=int(limit_volume*2)
#TargetOrder_Deliver3=int(limit_volume*1)
            

In [ ]:
##统计出上面两个之后算P_rate
P_rate=min(P_rate_cap,abs(Target_amount)/(Time_full_round*MarketBuyOrder_arrive_average_volume*MarketBuyOrder_arrive_rate_1min))#理论参与率

'''每一次想挂单的量'''

result_uncomplete_order={}
Result_Position={}

AGG_bid_newmode='off'
AGG_ask_newmode='off'


begin_bids_price5=[]
begin_asks_price5=[]

depth5_result = marketAPI.get_orderbook(spot_instrument_id, '5')      

#先做出begin_price5
for i in range(Best_Number):
    begin_bids_price5.append(precfloat(depth5_result['data'][0]['bids'][i][0],Tick_digit))
    begin_asks_price5.append(precfloat(depth5_result['data'][0]['asks'][i][0],Tick_digit))

while True:
    while True:
        try:
            #在资金费率开始前半小时跟后两小时机器人不要动,12600秒内不要动
            Nowtime=datetime.datetime.now()
            if Nowtime.hour == 7 or Nowtime.hour == 15 or Nowtime.hour == 23:
                if Nowtime.minute == 45 or Nowtime.minute == 46:
                    time.sleep(2700)
            time.sleep(0.2)
            judge_continue_mode='on'
            continue_maker_mode='off'
            continue_taker_mode='off'
            continue_taker_aggressive_mode='off'
            bids_new_price5=[]
            asks_new_price5=[]
            bids_p=[]
            asks_p=[]
            
            depth5_result = marketAPI.get_orderbook(spot_instrument_id, '5')  

            #做出new_price5, bids_p,asks_p
            for i in range(Best_Number):
                bids_new_price5.append(precfloat(depth5_result['data'][0]['bids'][i][0],Tick_digit))  #价格
                asks_new_price5.append(precfloat(depth5_result['data'][0]['asks'][i][0],Tick_digit))  #价格
                bids_p.append(depth5_result['data'][0]['bids'][i])            
                asks_p.append(depth5_result['data'][0]['asks'][i])            

            #begin跟new没变就不往下走
            if begin_bids_price5[:2]==bids_new_price5[:2]:
                if begin_asks_price5[:2]==asks_new_price5[:2]:
                   # continue_maker_mode='off'                
                    judge_continue_mode='off'
                    #time.sleep(0.1)       
                                       
            
            #best ask跟bid的价格差距,判断看是要挂单还是限价单        
            if judge_continue_mode=='on':
                c=precfloat(depth5_result['data'][0]['asks'][0][0],Tick_digit)-precfloat(depth5_result['data'][0]['bids'][0][0],Tick_digit)
                mode_type_index=precfloat(c/Tick_size,Tick_digit)
                if mode_type_index>=20*Tick_size:
                    continue_maker_mode='on'
                elif 2*Tick_size<mode_type_index<20*Tick_size:
                    continue_maker_mode='on'
                elif mode_type_index>=2*Tick_size:
                    continue_maker_mode='on'
                
            #print('continue_maker_mode',continue_maker_mode)
            #print('continue_taker_mode',continue_taker_mode)
            #print('continue_taker_aggressive_mode',continue_taker_aggressive_mode)                           
                                        
                    
            if continue_maker_mode=='on':
                #这里是uncomplete订单
               # result_uncomplete_order=swapAPI.get_order_list(instrument_id=spot_instrument_id, state='6', after='', before='', limit='')           
                uncomplete_order_result = tradeAPI.get_order_list(instId=spot_instrument_id)         

                AGG_bid_newmode='off'   #调仓模式-bid
                bid_cancel_mode='on'
                AGG_ask_newmode='off'   #调仓模式-ask
                ask_cancel_mode='on'

                mode={'AGG_position_close_mode':[AGG_bid_newmode,AGG_ask_newmode],     
                      'cancel_mode_type':[bid_cancel_mode,ask_cancel_mode],
                     'begin_price5':[begin_bids_price5,begin_asks_price5],
                      'bid_ask':[bids_p,asks_p],
                      'opp_bid_ask':[asks_p,bids_p],
                     'label':['BAG1','AAG1'],
                     'number_order_take_decide':'1'}                                        

                nb_mode=bid_ask_mode_switch('bid',mode)                                        
                na_mode=bid_ask_mode_switch('ask',mode)

                try:
                    mode['AGG_position_close_mode'][0],mode['cancel_mode_type'][0]=close_position_judge(nb_mode,result_uncomplete_order)
                    mode['AGG_position_close_mode'][1],mode['cancel_mode_type'][1]=close_position_judge(na_mode,result_uncomplete_order)
                except:
                    pass

                #开始合成出new price跟volume,看AGG_newmode
                bids_new_price5=[]
                bids_new_volume5=[]
                if AGG_bid_newmode=='on':
                    for i in range(1,Best_Number+1):  
                        bids_new_price5.append(precfloat(bids_p[i][0],Tick_digit))
                        bids_new_volume5.append(int(float(bids_p[i][1])))

                elif AGG_bid_newmode=='off':
                    for i in range(Best_Number):                                         
                        bids_new_price5.append(precfloat(bids_p[i][0],Tick_digit))
                        bids_new_volume5.append(int(float(bids_p[i][1])))

                bids_new_volume5_arr = np.array(bids_new_volume5)
                asks_new_price5=[]
                asks_new_volume5=[]

                if AGG_ask_newmode=='on':  
                    for i in range(1,Best_Number+1):                                       
                        asks_new_price5.append(precfloat(asks_p[i][0],Tick_digit))
                        asks_new_volume5.append(int(float(asks_p[i][1])))
                elif AGG_ask_newmode=='off':

                    for i in range(Best_Number):                                       
                        asks_new_price5.append(precfloat(asks_p[i][0],Tick_digit))
                        asks_new_volume5.append(int(float(asks_p[i][1])))
                asks_new_volume5_arr = np.array(asks_new_volume5)

                #如果5档都没改变，跳出去，这边要小心，5档没变不代表没成交
                if begin_bids_price5[:2]==bids_new_price5[:2]:  
                    bid_mode='off'
                if begin_asks_price5[:2]==asks_new_price5[:2]:
                    ask_mode='off'
                if begin_bids_price5[:2]!=bids_new_price5[:2]:
                    bid_mode='on'
                if begin_asks_price5[:2]!=asks_new_price5[:2]:
                    ask_mode='on'   

                if bid_mode=='off':
                    if ask_mode=='off':
                        break

               # quantity_limit= min((TargetOrder_Deliver0+TargetOrder_Deliver1+limit_volume)*2,50)
                quantity_limit= 2000

               #Result_Position= swapAPI.get_specific_position(spot_instrument_id)['holding']
                Result_Position =accountAPI.get_account(use_currency)['data']   
                #仓位,带方向(期权仓位单位为BTC, 期货仓位单位为USD),正数为多仓，负数为空仓。


                BAGMODE='off' #Bid-Aggrassive-mode,打开就要下中间价
                AAGMODE='off'
                try:
                    BAGMODE,AAGMODE,bid_mode,ask_mode,LSdiff=Indentify_Position_Premiunm(Result_Position,spot_instrument_id,
                                                                                  quantity_limit,BAGMODE,AAGMODE,bid_mode,ask_mode,base)
                except:
                    pass             

                '''计算ask/bid挂单价'''

                mode={'P_rate':P_rate,
                    'MarketOrder_arrive_average_volume':MarketBuyOrder_arrive_average_volume,
                    'limit_volume':limit_volume,
                    'Tick_size':Tick_size,      
                    'new_volume5_arr':bids_new_volume5_arr,     #这是array，宣告位置错误
                    'new_price5':bids_new_price5,
                    'opp_new_price5':asks_new_price5,          #这是list, opposite
                    'bidask_index':1,                          #bid=1,ask=-1
                    'Tick_digit':Tick_digit,                            #tick_digit,交易价格到的最小位数   
                    'ask_bid_pricegap':2,                     #best bid/ask中间有几个tick
                    'niddle_priceorder_times':niddle_priceorder_times,              #最远下单跟次远下单中间差几个tieck
                     'TargetOrder_Deliver0':TargetOrder_Deliver0,
                     'TargetOrder_Deliver1':TargetOrder_Deliver1}
                   #   'TargetOrder_Deliver2':TargetOrder_Deliver2,
                   #   'TargetOrder_Deliver3':TargetOrder_Deliver3} 


               # print('bid_mode',bid_mode)
               # print('ask_mode',ask_mode)
               # print('AGGMODE',AAGMODE)
               # print('BAGMODE',BAGMODE)
               # print('LSdiff',LSdiff)


                if bid_mode=='on':
                    mode['bidask_index']=1
                    mode['new_volume5_arr']=bids_new_volume5_arr
                    mode['new_price5']=bids_new_price5
                    mode['opp_new_price5']=asks_new_price5
                    bid_candidate_list=xiadan_list(mode,Best_Number)    #产生bid下单的candidate list
                    #print('bid_candidate_list_1',bid_candidate_list)

                if ask_mode=='on':
                    mode['bidask_index']=-1
                    mode['new_volume5_arr']=asks_new_volume5_arr
                    mode['new_price5']=asks_new_price5
                    mode['opp_new_price5']=bids_new_price5
                    ask_candidate_list=xiadan_list(mode,Best_Number)    #产生ask下单的candidate list
                    #print('ask_candidate_list',ask_candidate_list)

                '''开始计算uncomplete的单子'''

                uncomplete_order_result = tradeAPI.get_order_list(instId=spot_instrument_id)   

                order_cancel_list=[]
                if len(uncomplete_order_result ['data']) !=0:
                    A=uncomplete_order_result ['data']
                    for i in A:
                        order_cancel_list.append(i)
                    idd=[]
                    for i in order_cancel_list:
                        B={"instId": spot_instrument_id, "ordId": i['ordId']}
                        idd.append(B)  
                    if idd != []:
                        result = tradeAPI.cancel_multiple_orders(idd) 

                '''bid aggressive mode'''  #long>short 所以要平多
                    #因为order_id不好单独提取,所以这里设client_oid
                    #LSdiff是持仓位差,float,是正数
                    #print('LSdiff',LSdiff)

               # print('BAGMODE_1',BAGMODE)
               # print('AAGMODE_1',AAGMODE)

                take_orders_list=[]
                if BAGMODE =='on':          #bid端的agg模式,ask要挂远,bid要挂近  short>long  平空 buy
                    mapping={}
                    value_list=[]
                    key_list=['instId','tdMode','side','px','sz',"ordType"]                    
                    for p in range(round((LSdiff-quantity_limit)/limit_volume)):   #持仓差差了几个qty,然后挂几次平空单
                        LtoSmiddle=str(precfloat(bids_new_price5[0]+pow(-1, p)*Tick_size,Tick_digit))  #挂bid best  price
                        for key, value in zip(key_list, [spot_instrument_id,'cross','buy',LtoSmiddle,str(limit_volume),"limit"]):  #普通委托,不是挂单,没加label, qty先以100为单位
                            mapping[key]=value
                        take_orders_list.append(eval(json.dumps(mapping)))

                '''ask aggressive mode'''  #Long>short                              
                if AAGMODE =='on':              #ask端的agg模式,bid要挂远,ask要挂近  平多 sell
                    mapping={}
                    value_list=[]
                    key_list=['instId','tdMode','side','px','sz',"ordType"]                
                    for p in range(round((LSdiff-quantity_limit)/limit_volume)):   #持仓差差了几个qty,然后挂几次平空单
                        StoLmiddle=str(precfloat(asks_new_price5[0]-pow(-1, p)*Tick_size,Tick_digit))   #挂ask best
                        for key, value in zip(key_list, [spot_instrument_id,'cross','sell',StoLmiddle,str(limit_volume),"limit"]):  #普通委托,不是挂单,没加label,qty先以100为单位
                            mapping[key]=value
                        take_orders_list.append(eval(json.dumps(mapping)))


                '''下单'''       
                if bid_mode=='on':
                  #  mapping={}
                  #  value_list=[]
                  #  take_orders_list=[]
                    b=[]
                  #  print('len(bid_candidate_list)',len(bid_candidate_list))
                    for i in range(len(bid_candidate_list)):
                        key_list=['instId','tdMode','side','px','sz',"ordType"]
                        b=dict(zip(key_list, [spot_instrument_id,'cross','buy', str(bid_candidate_list[i][0]),str(bid_candidate_list[i][1]),"post_only"]))
                        take_orders_list.append(b)

                   # for i in range(len(bid_candidate_list)):
                    #    key_list=['instrument_id','side','price','qty','post_only',"order_type"]
                    #    for key, value in zip(key_list, [spot_instrument_id,'buy', str(bid_candidate_list[i][0]),str(bid_candidate_list[i][1]),True,"limit"]):
                    #        mapping[key]=value
                    #    take_orders_list.append(mapping)


                if ask_mode=='on':
                   # mapping={}
                   # value_list=[]
                    b=[]
                    #print('len(ask_candidate_list)',len(ask_candidate_list))
                    for i in range(len(ask_candidate_list)):
                        key_list=['instId','tdMode','side','px','sz',"ordType"]
                        b=dict(zip(key_list, [spot_instrument_id,'cross','sell',str(ask_candidate_list[i][0]),str(ask_candidate_list[i][1]),"post_only"]))
                        take_orders_list.append(b)
                        #print('take_orders_list_ask',take_orders_list)


                '''下单'''
                #print('take_orders_list_2',take_orders_list)
                if take_orders_list!=[]:    
                    order_result = tradeAPI.place_multiple_orders(take_orders_list)
                #print('order_result',order_result)

                    #print('A',A)
                '''重新记录初始五档'''
                begin_bids_price5=[]
                for i in range(Best_Number):
                    begin_bids_price5.append(bids_new_price5[i])

                begin_asks_price5=[]
                for i in range(Best_Number):
                    begin_asks_price5.append(asks_new_price5[i])
    
        
                      
        except Exception as e:
            time_1 = get_timestamp()
            print(time_1 + "连接断开，正在重连……")
            print(e)




In [ ]:
if continue_taker_mode=='on':
    #这里是uncomplete订单
   # result_uncomplete_order=swapAPI.get_order_list(instrument_id=spot_instrument_id, state='6', after='', before='', limit='')           
    uncomplete_order_result = tradeAPI.get_order_list(instId=spot_instrument_id)      

    AGG_bid_newmode='off'   #调仓模式-bid
    bid_cancel_mode='on'
    AGG_ask_newmode='off'   #调仓模式-ask
    ask_cancel_mode='on'

    mode={'AGG_position_close_mode':[AGG_bid_newmode,AGG_ask_newmode],     
          'cancel_mode_type':[bid_cancel_mode,ask_cancel_mode],
         'begin_price5':[begin_bids_price5,begin_asks_price5],
          'bid_ask':[bids_p,asks_p],
          'opp_bid_ask':[asks_p,bids_p],
         'label':['BAG1','AAG1'],
         'number_order_take_decide':'1'}                                        

    nb_mode=bid_ask_mode_switch('bid',mode)                                        
    na_mode=bid_ask_mode_switch('ask',mode)

    try:
        mode['AGG_position_close_mode'][0],mode['cancel_mode_type'][0]=close_position_judge(nb_mode,result_uncomplete_order)
        mode['AGG_position_close_mode'][1],mode['cancel_mode_type'][1]=close_position_judge(na_mode,result_uncomplete_order)
    except:
        pass

    #开始合成出new price跟volume,看AGG_newmode
    bids_new_price5=[]
    bids_new_volume5=[]
    if AGG_bid_newmode=='on':
        for i in range(1,Best_Number+1):  
            bids_new_price5.append(precfloat(bids_p[i][0],Tick_digit))
            bids_new_volume5.append(int(float(bids_p[i][1])))

    elif AGG_bid_newmode=='off':
        for i in range(Best_Number):                                         
            bids_new_price5.append(precfloat(bids_p[i][0],Tick_digit))
            bids_new_volume5.append(int(float(bids_p[i][1])))

    bids_new_volume5_arr = np.array(bids_new_volume5)

    asks_new_price5=[]
    asks_new_volume5=[]
    if AGG_ask_newmode=='on':  
        for i in range(1,Best_Number+1):                                       
            asks_new_price5.append(precfloat(asks_p[i][0],Tick_digit))
            asks_new_volume5.append(int(float(asks_p[i][1])))
    elif AGG_ask_newmode=='off':

        for i in range(Best_Number):                                       
            asks_new_price5.append(precfloat(asks_p[i][0],Tick_digit))
            asks_new_volume5.append(int(float(asks_p[i][1])))
    asks_new_volume5_arr = np.array(asks_new_volume5)

    #如果5档都没改变，跳出去，这边要小心，5档没变不代表没成交
    if begin_bids_price5[:2]==bids_new_price5[:2]:  
        bid_mode='off'
    if begin_asks_price5[:2]==asks_new_price5[:2]:
        ask_mode='off'
    if begin_bids_price5[:2]!=bids_new_price5[:2]:
        bid_mode='on'
    if begin_asks_price5[:2]!=asks_new_price5[:2]:
        ask_mode='on'   

    if bid_mode=='off':
        if ask_mode=='off':
            break

    quantity_limit= min((TargetOrder_Deliver0+TargetOrder_Deliver1+limit_volume)*2,30)

   #Result_Position= swapAPI.get_specific_position(spot_instrument_id)['holding']

    Result_Position =accountAPI.get_account(use_currency)['data'] 
    #仓位,带方向(期权仓位单位为BTC, 期货仓位单位为USD),正数为多仓，负数为空仓。


    BAGMODE='off' #Bid-Aggrassive-mode,打开就要下中间价
    AAGMODE='off'
    try:
        BAGMODE,AAGMODE,bid_mode,ask_mode,LSdiff=Indentify_Position_Premiunm(Result_Position,spot_instrument_id,
                                                                      quantity_limit,BAGMODE,AAGMODE,bid_mode,ask_mode,base)
    except:
        pass

    '''计算ask/bid挂单价'''

    mode={'P_rate':P_rate,
        'MarketOrder_arrive_average_volume':MarketBuyOrder_arrive_average_volume,
        'limit_volume':limit_volume,
        'Tick_size':Tick_size,      
        'new_volume5_arr':bids_new_volume5_arr,     #这是array，宣告位置错误
        'new_price5':bids_new_price5,
        'opp_new_price5':asks_new_price5,          #这是list, opposite
        'bidask_index':1,                          #bid=1,ask=-1
        'Tick_digit':Tick_digit,                            #tick_digit,交易价格到的最小位数   
        'ask_bid_pricegap':2,                     #best bid/ask中间有几个tick
        'niddle_priceorder_times':niddle_priceorder_times,              #最远下单跟次远下单中间差几个tieck
         'TargetOrder_Deliver0':TargetOrder_Deliver0,
         'TargetOrder_Deliver1':TargetOrder_Deliver1}
       #   'TargetOrder_Deliver2':TargetOrder_Deliver2,
       #   'TargetOrder_Deliver3':TargetOrder_Deliver3} 


    #print('bid_mode',bid_mode)
    #print('ask_mode',ask_mode)
    #print('AGGMODE',AAGMODE)
    #print('BAGMODE',BAGMODE)
    #print('LSdiff',LSdiff)


    if bid_mode=='on':
        mode['bidask_index']=1
        mode['new_volume5_arr']=bids_new_volume5_arr
        mode['new_price5']=bids_new_price5
        mode['opp_new_price5']=asks_new_price5
        bid_candidate_list=xiadan_list(mode,Best_Number)    #产生bid下单的candidate list
        #print('bid_candidate_list_1',bid_candidate_list)

    if ask_mode=='on':
        mode['bidask_index']=-1
        mode['new_volume5_arr']=asks_new_volume5_arr
        mode['new_price5']=asks_new_price5
        mode['opp_new_price5']=bids_new_price5
        ask_candidate_list=xiadan_list(mode,Best_Number)    #产生ask下单的candidate list
       # print('ask_candidate_list',ask_candidate_list)

    '''开始计算uncomplete的单子'''

    uncomplete_order_result = tradeAPI.get_order_list(instId=spot_instrument_id)   

    order_cancel_list=[]
    if len(uncomplete_order_result ['data'])!=0:
        A=uncomplete_order_result ['data']
        for i in A:
            order_cancel_list.append(i)
        idd=[]
        for i in order_cancel_list:
            B={"instId": spot_instrument_id, "ordId": i['ordId']}
            idd.append(B)  
        if idd != []:
            result = tradeAPI.cancel_multiple_orders(idd)

    '''bid aggressive mode'''  #long>short 所以要平多
        #因为order_id不好单独提取,所以这里设client_oid
        #LSdiff是持仓位差,float,是正数
        #print('LSdiff',LSdiff)

    #print('BAGMODE',BAGMODE)
    #print('AAGMODE',AAGMODE)

    take_orders_list=[]
    if BAGMODE =='on':          #bid端的agg模式,ask要挂远,bid要挂近  short>long  平空 buy
        mapping={}
        value_list=[]
        key_list=['instId','tdMode','side','px','sz',"ordType"]   

        for p in range(round(LSdiff-quantity_limit)):   #持仓差差了几个qty,然后挂几次平空单

            LtoSmiddle=str(precfloat(bids_new_price5[0]+pow(-1, p)*Tick_size,Tick_digit))  #挂bid best  price

            for key, value in zip(key_list, [spot_instrument_id,'cross','buy',LtoSmiddle,str(limit_volume*2),"limit"]):  #普通委托,不是挂单,没加label, qty先以100为单位
                mapping[key]=value
            take_orders_list.append(eval(json.dumps(mapping)))



    '''ask aggressive mode'''  #Long>short                              
    if AAGMODE =='on':              #ask端的agg模式,bid要挂远,ask要挂近  平多 sell
        mapping={}
        value_list=[]
        key_list=['instId','tdMode','side','px','sz',"ordType"]                
        for p in range(round(LSdiff-quantity_limit)):   #持仓差差了几个qty,然后挂几次平空单
            StoLmiddle=str(precfloat(asks_new_price5[0]-pow(-1, p)*Tick_size,Tick_digit))   #挂ask best
            for key, value in zip(key_list, [spot_instrument_id,'cross','sell',StoLmiddle,str(limit_volume*2),"limit"]):  #普通委托,不是挂单,没加label,qty先以100为单位
                mapping[key]=value
            take_orders_list.append(eval(json.dumps(mapping)))


    '''下单'''       
    if bid_mode=='on':
      #  mapping={}
      #  value_list=[]
      #  take_orders_list=[]
        b=[]
        #print('len(bid_candidate_list)',len(bid_candidate_list))
        for i in range(len(bid_candidate_list)):
            key_list=['instId','tdMode','side','px','sz',"ordType"]
            b=dict(zip(key_list, [spot_instrument_id,'cross','buy', str(bid_candidate_list[i][0]),str(bid_candidate_list[i][1]),"post_only"]))
            take_orders_list.append(b)

       # for i in range(len(bid_candidate_list)):
        #    key_list=['instrument_id','side','price','qty','post_only',"order_type"]
        #    for key, value in zip(key_list, [spot_instrument_id,'buy', str(bid_candidate_list[i][0]),str(bid_candidate_list[i][1]),True,"limit"]):
        #        mapping[key]=value
        #    take_orders_list.append(mapping)


    if ask_mode=='on':
       # mapping={}
       # value_list=[]
        b=[]
        #print('len(ask_candidate_list)',len(ask_candidate_list))
        for i in range(len(ask_candidate_list)):
            key_list=['instId','tdMode','side','px','sz',"ordType"]
            b=dict(zip(key_list, [spot_instrument_id,'cross','sell', str(ask_candidate_list[i][0]),str(ask_candidate_list[i][1]),"post_only"]))
            take_orders_list.append(b)
            #print('take_orders_list_ask',take_orders_list)


    '''下单'''
    #print('take_orders_list',take_orders_list)
    if take_orders_list!=[]:    
        order_result = tradeAPI.place_multiple_orders(take_orders_list)
    #print('order_result',order_result)

        #print('A',A)
    '''重新记录初始五档'''
    begin_bids_price5=[]
    for i in range(Best_Number):
        begin_bids_price5.append(bids_new_price5[i])

    begin_asks_price5=[]
    for i in range(Best_Number):
        begin_asks_price5.append(asks_new_price5[i])

In [ ]:
if continue_taker_aggressive_mode=='on':
    #这里是uncomplete订单
   # result_uncomplete_order=swapAPI.get_order_list(instrument_id=spot_instrument_id, state='6', after='', before='', limit='')           
    uncomplete_order_result = tradeAPI.get_order_list(instId=spot_instrument_id)        

    AGG_bid_newmode='off'   #调仓模式-bid
    bid_cancel_mode='on'
    AGG_ask_newmode='off'   #调仓模式-ask
    ask_cancel_mode='on'

    mode={'AGG_position_close_mode':[AGG_bid_newmode,AGG_ask_newmode],     
          'cancel_mode_type':[bid_cancel_mode,ask_cancel_mode],
         'begin_price5':[begin_bids_price5,begin_asks_price5],
          'bid_ask':[bids_p,asks_p],
          'opp_bid_ask':[asks_p,bids_p],
         'label':['BAG1','AAG1'],
         'number_order_take_decide':'1'}                                        

    nb_mode=bid_ask_mode_switch('bid',mode)                                        
    na_mode=bid_ask_mode_switch('ask',mode)

    try:
        mode['AGG_position_close_mode'][0],mode['cancel_mode_type'][0]=close_position_judge(nb_mode,result_uncomplete_order)
        mode['AGG_position_close_mode'][1],mode['cancel_mode_type'][1]=close_position_judge(na_mode,result_uncomplete_order)
    except:
        pass

    #开始合成出new price跟volume,看AGG_newmode
    bids_new_price5=[]
    bids_new_volume5=[]
    if AGG_bid_newmode=='on':
        for i in range(1,Best_Number+1):  
            bids_new_price5.append(precfloat(bids_p[i][0],Tick_digit))
            bids_new_volume5.append(int(float(bids_p[i][1])))

    elif AGG_bid_newmode=='off':
        for i in range(Best_Number):                                         
            bids_new_price5.append(precfloat(bids_p[i][0],Tick_digit))
            bids_new_volume5.append(int(float(bids_p[i][1])))

    bids_new_volume5_arr = np.array(bids_new_volume5)

    asks_new_price5=[]
    asks_new_volume5=[]

    if AGG_ask_newmode=='on':  
        for i in range(1,Best_Number+1):                                       
            asks_new_price5.append(precfloat(asks_p[i][0],Tick_digit))
            asks_new_volume5.append(int(float(asks_p[i][1])))
    elif AGG_ask_newmode=='off':

        for i in range(Best_Number):                                       
            asks_new_price5.append(precfloat(asks_p[i][0],Tick_digit))
            asks_new_volume5.append(int(float(asks_p[i][1])))
    asks_new_volume5_arr = np.array(asks_new_volume5)

    #如果5档都没改变，跳出去，这边要小心，5档没变不代表没成交
    if begin_bids_price5[:2]==bids_new_price5[:2]:  
        bid_mode='off'
    if begin_asks_price5[:2]==asks_new_price5[:2]:
        ask_mode='off'
    if begin_bids_price5[:2]!=bids_new_price5[:2]:
        bid_mode='on'
    if begin_asks_price5[:2]!=asks_new_price5[:2]:
        ask_mode='on'   

    if bid_mode=='off':
        if ask_mode=='off':
            break

    quantity_limit= min((TargetOrder_Deliver0+TargetOrder_Deliver1+limit_volume)*2,50)

   #Result_Position= swapAPI.get_specific_position(spot_instrument_id)['holding']    
    Result_Position =accountAPI.get_account(use_currency)['data']  
    #仓位,带方向(期权仓位单位为BTC, 期货仓位单位为USD),正数为多仓，负数为空仓。


    BAGMODE='off' #Bid-Aggrassive-mode,打开就要下中间价
    AAGMODE='off'
    try:
        BAGMODE,AAGMODE,bid_mode,ask_mode,LSdiff=Indentify_Position_Premiunm(Result_Position,spot_instrument_id,
                                                                      quantity_limit,BAGMODE,AAGMODE,bid_mode,ask_mode,base)
    except:
        pass

    '''计算ask/bid挂单价'''

    mode={'P_rate':P_rate,
        'MarketOrder_arrive_average_volume':MarketBuyOrder_arrive_average_volume,
        'limit_volume':limit_volume,
        'Tick_size':Tick_size,      
        'new_volume5_arr':bids_new_volume5_arr,     #这是array，宣告位置错误
        'new_price5':bids_new_price5,
        'opp_new_price5':asks_new_price5,          #这是list, opposite
        'bidask_index':1,                          #bid=1,ask=-1
        'Tick_digit':Tick_digit,                            #tick_digit,交易价格到的最小位数   
        'ask_bid_pricegap':2,                     #best bid/ask中间有几个tick
        'niddle_priceorder_times':niddle_priceorder_times,              #最远下单跟次远下单中间差几个tieck
         'TargetOrder_Deliver0':TargetOrder_Deliver0,
         'TargetOrder_Deliver1':TargetOrder_Deliver1}
       #   'TargetOrder_Deliver2':TargetOrder_Deliver2,
       #   'TargetOrder_Deliver3':TargetOrder_Deliver3} 


    #print('bid_mode',bid_mode)
    #print('ask_mode',ask_mode)
    #print('AGGMODE',AAGMODE)
    #print('BAGMODE',BAGMODE)
    #print('LSdiff',LSdiff)


    if bid_mode=='on':
        mode['bidask_index']=1
        mode['new_volume5_arr']=bids_new_volume5_arr
        mode['new_price5']=bids_new_price5
        mode['opp_new_price5']=asks_new_price5
        bid_candidate_list=xiadan_list(mode,Best_Number)    #产生bid下单的candidate list
       # print('bid_candidate_list',bid_candidate_list)

    if ask_mode=='on':
        mode['bidask_index']=-1
        mode['new_volume5_arr']=asks_new_volume5_arr
        mode['new_price5']=asks_new_price5
        mode['opp_new_price5']=bids_new_price5
        ask_candidate_list=xiadan_list(mode,Best_Number)    #产生ask下单的candidate list
        #print('ask_candidate_list',ask_candidate_list)

    '''开始计算uncomplete的单子'''

    uncomplete_order_result = tradeAPI.get_order_list(instId=spot_instrument_id)   

    order_cancel_list=[]
    if len(uncomplete_order_result ['data']) !=0:
        A=uncomplete_order_result ['data']
        for i in A:
            order_cancel_list.append(i)
        idd=[]
        for i in order_cancel_list:
            B={"instId": spot_instrument_id, "ordId": i['ordId']}
            idd.append(B)  
        if idd != []:
            result = tradeAPI.cancel_multiple_orders(idd)

    '''bid aggressive mode'''  #long>short 所以要平多
        #因为order_id不好单独提取,所以这里设client_oid
        #LSdiff是持仓位差,float,是正数
        #print('LSdiff',LSdiff)

    #print('BAGMODE',BAGMODE)
    #print('AAGMODE',AAGMODE)

    take_orders_list=[]
    if BAGMODE =='on':          #bid端的agg模式,ask要挂远,bid要挂近  short>long  平空 buy
        mapping={}
        value_list=[]
        key_list=['instId','tdMode','side','px','sz',"ordType"]                    
        for p in range(round(LSdiff-quantity_limit)):   #持仓差差了几个qty,然后挂几次平空单
            LtoSmiddle=str(precfloat(bids_new_price5[0]+pow(-1, p)*Tick_size,Tick_digit))  #挂bid best  price
            for key, value in zip(key_list, [spot_instrument_id,'cross','buy',LtoSmiddle,str(limit_volume*2),"limit"]):  #普通委托,不是挂单,没加label, qty先以100为单位
                mapping[key]=value
            take_orders_list.append(eval(json.dumps(mapping)))

    '''ask aggressive mode'''  #Long>short                              
    if AAGMODE =='on':              #ask端的agg模式,bid要挂远,ask要挂近  平多 sell
        mapping={}
        value_list=[]
        key_list=['instId','tdMode','side','px','sz',"ordType"]                
        for p in range(round(LSdiff-quantity_limit)):   #持仓差差了几个qty,然后挂几次平空单
            StoLmiddle=str(precfloat(asks_new_price5[0]-pow(-1, p)*Tick_size,Tick_digit))   #挂ask best
            for key, value in zip(key_list, [spot_instrument_id,'cross','sell',StoLmiddle,str(limit_volume*2),"limit"]):  #普通委托,不是挂单,没加label,qty先以100为单位
                mapping[key]=value
            take_orders_list.append(eval(json.dumps(mapping)))


    '''下单'''       
    if bid_mode=='on':
      #  mapping={}
      #  value_list=[]
      #  take_orders_list=[]
        b=[]
        #print('len(bid_candidate_list)',len(bid_candidate_list))
        for i in range(len(bid_candidate_list)):
            key_list=['instId','tdMode','side','px','sz',"ordType"]
            b=dict(zip(key_list, [spot_instrument_id,'cross','buy', str(ask_candidate_list[i][0]),str(bid_candidate_list[i][1]),"limit"]))
            take_orders_list.append(b)

       # for i in range(len(bid_candidate_list)):
        #    key_list=['instrument_id','side','price','qty','post_only',"order_type"]
        #    for key, value in zip(key_list, [spot_instrument_id,'buy', str(bid_candidate_list[i][0]),str(bid_candidate_list[i][1]),True,"limit"]):
        #        mapping[key]=value
        #    take_orders_list.append(mapping)

    if ask_mode=='on':
       # mapping={}
       # value_list=[]
        b=[]
        #print('len(ask_candidate_list)',len(ask_candidate_list))
        for i in range(len(ask_candidate_list)):
            key_list=['instId','tdMode','side','px','sz',"ordType"]
            b=dict(zip(key_list, [spot_instrument_id,'cross','sell', str(bid_candidate_list[i][0]),str(ask_candidate_list[i][1]),"limit"]))
            take_orders_list.append(b)
            #print('take_orders_list_ask',take_orders_list)


    '''下单'''
    #print('take_orders_list',take_orders_list)
    if take_orders_list!=[]:    
        order_result = tradeAPI.place_multiple_orders(take_orders_list)
    #print('order_result',order_result)
    
    
    
      #print('A',A)
    '''重新记录初始五档'''
    begin_bids_price5=[]
    for i in range(Best_Number):
        begin_bids_price5.append(bids_new_price5[i])

    begin_asks_price5=[]
    for i in range(Best_Number):
        begin_asks_price5.append(asks_new_price5[i])